In [26]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy --user
!{sys.executable} -m pip install pandas --user
!{sys.executable} -m pip install scikit-learn --user
!{sys.executable} -m pip install matplotlib --user
!{sys.executable} -m pip install plotly --user

NameError: name 'init_notebook_mode' is not defined

load data

In [10]:
import pandas as pd

# making data frame from csv file 
data = pd.read_csv("../data/NACC.csv", index_col ="NACCID" )

#print number of columns
print("Original data set has %d samples and %d variables" %(data.shape[0], data.shape[1]))
  
# dropping passed columns 
data.drop(["NACCNEC", "NACCWRI1", "NACCWRI2", "NACCWRI3", "NACCINT", "NACCPICK","NACCCBD","NACCPROG", "NACCADC", "NACCMOD", "NACCYOD", "NACCAUTP", "NPFORMVER"], axis = 1, inplace = True) 

# clean data
data = data.query('NACCBRAA <= 7')

#print number of columns
print("After drop data set has %d samples and %d variables" %(data.shape[0], data.shape[1]))

# display
data

Original data set has 5636 samples and 30 variables
After drop data set has 5636 samples and 17 variables


,NPSEX,NACCNEUR,NACCDIFF,NACCVASC,NACCAMY,NACCINF,NACCHEM,NACCARTE,NACCLEWY,NACCPICK+,NACCCBD+,NACCPROG+,NACCPRIO,NACCDOWN,NACCOTHP,NACCDAGE,NACCBRAA
NACCID,,,,,,,,,,,,,,,,,
NACC000385,1,2,3,1,2,1,9,1,0,0,0,0,0,7,0,89,4
NACC000546,1,3,8,1,0,0,0,0,3,0,0,0,0,7,0,81,6
NACC000579,2,1,1,1,0,0,0,3,0,0,0,0,0,7,0,93,3
NACC000618,1,3,3,1,2,0,0,3,2,0,0,0,0,7,0,72,6
NACC000731,2,0,3,1,0,1,0,3,0,0,0,0,0,7,0,83,2
NACC001008,1,2,3,1,3,0,0,8,0,0,0,0,0,7,0,91,4
NACC001723,1,1,1,1,0,1,0,2,0,0,0,0,0,7,0,103,4
NACC002337,2,1,1,1,0,0,0,8,0,0,0,0,0,7,1,100,5
NACC002673,2,2,3,1,2,0,0,0,0,8,8,8,0,7,0,83,5


Prepare data

In [16]:
from sklearn.model_selection import train_test_split
X = data.values[:, :-2]
Y_age = data.values[:, -2] # age of death
Y_bra = data.values[:, -1] # braak stage

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(X, Y_bra, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
depths = [2, 3, 4, 5, 6, 10, 100, 1000]
tuned_parameters = [{'max_depth': depths}] # max tree deepth
scores = ['neg_mean_absolute_error', 'neg_mean_squared_error']

Run CV

In [18]:
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
best_index = -1
best_clf = None
for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(tree.DecisionTreeRegressor(), tuned_parameters, cv=5,
                           scoring=score)
        clf.fit(X_train, y_train)

        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print()

        # save deepest tree found between estimators
        if clf.best_index_ > best_index:
            best_index = clf.best_index_
            best_clf = clf.best_estimator_

# Tuning hyper-parameters for neg_mean_absolute_error

Best parameters set found on development set:

{'max_depth': 4}

Grid scores on development set:

-1.099 (+/-0.053) for {'max_depth': 2}
-1.055 (+/-0.061) for {'max_depth': 3}
-1.042 (+/-0.046) for {'max_depth': 4}
-1.045 (+/-0.042) for {'max_depth': 5}
-1.064 (+/-0.036) for {'max_depth': 6}
-1.119 (+/-0.066) for {'max_depth': 10}
-1.232 (+/-0.089) for {'max_depth': 100}
-1.226 (+/-0.143) for {'max_depth': 1000}

# Tuning hyper-parameters for neg_mean_squared_error

Best parameters set found on development set:

{'max_depth': 3}

Grid scores on development set:

-2.157 (+/-0.285) for {'max_depth': 2}
-2.024 (+/-0.295) for {'max_depth': 3}
-2.046 (+/-0.189) for {'max_depth': 4}
-2.074 (+/-0.222) for {'max_depth': 5}
-2.194 (+/-0.160) for {'max_depth': 6}
-2.595 (+/-0.293) for {'max_depth': 10}
-3.451 (+/-0.513) for {'max_depth': 100}
-3.495 (+/-0.418) for {'max_depth': 1000}



Get best model results

In [21]:
    import numpy as np
    y_true, y_pred = y_test, best_clf.predict(X_test)
    y_pred_ceil = np.ceil(y_pred).astype(int)  # ceiling melhorou 15% a acc
    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    print(classification_report(y_true, y_pred_ceil))
    print()
    print()

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       174
           1       0.03      0.00      0.01       209
           2       0.27      0.50      0.35       306
           3       0.23      0.12      0.16       285
           4       0.27      0.25      0.26       390
           5       0.26      0.26      0.26       498
           6       0.60      0.79      0.68       914
           7       0.00      0.00      0.00        20
           8       0.00      0.00      0.00        18
           9       0.00      0.00      0.00         4

    accuracy                           0.40      2818
   macro avg       0.17      0.19      0.17      2818
weighted avg       0.33      0.40      0.36      2818





/home/adamski/.local/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Plot Results

In [29]:
    import matplotlib.pyplot as plt
    from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
    from plotly.tools import mpl_to_plotly
    
    init_notebook_mode(connected=True)
    
    fig, axes = plt.subplots(nrows=1, ncols=2)
    ax0, ax1 = axes.flatten()
    ax0.hist(y_test, 8, density=True, histtype='bar')
    ax0.set_title('Original')

    ax1.hist(y_pred, 8)
    ax1.set_title('Predicted')

    fig.tight_layout()
    plotly_fig = mpl_to_plotly( fig )
    iplot(plotly_fig, filename='mpl-basic-histogram')